# Propane Tutorial

Created by Davy Yue 2017-06-14


### Imports

In [ ]:
import itertools
import string
import os

import numpy as np

from msibi import MSIBI, State, Pair, mie

### Remove files generated during CG simulation

In [ ]:
os.system('rm rdfs/pair_C3*_state*-step*.txt f_fits.log')
os.system('rm state_*/*.txt state*/run.py state*/*query.dcd')

### Set up global parameters
Cutoff radius set to 5.0 units. Parameters including number of data points and potential cutoff are passed to `MSIBI`.

In [ ]:
rdf_cutoff = 5.0
opt = MSIBI(rdf_cutoff=rdf_cutoff, n_rdf_points=201, pot_cutoff=3.0,
        smooth_rdfs=True)

### Specify states
States each are initialized with different temperatures, directories, and start.hoomdxml files. 

A list `states` contains all the individual states: `stateA`, `stateB`, `stateC`.

In [ ]:
stateA = State(kT=0.5, state_dir='./state_A', top_file='start.hoomdxml',
               name='stateA', backup_trajectory=True)
stateB = State(kT=1.5, state_dir='./state_B', top_file='start.hoomdxml',
               name='stateB', backup_trajectory=True)
stateC = State(kT=2.0, state_dir='./state_C', top_file='start.hoomdxml',
               name='stateC', backup_trajectory=True)
states = [stateA, stateB, stateC]

### Specify pairs

Creates a list of all the possible indices for the 1024 atoms. 

Passes the type of interaction to be optimized, a `C3` to itself, to `Pair`. Sets the alpha values to `1.0`

In [ ]:
indices = list(itertools.combinations(range(1024), 2))  # all-all for 1024 atoms

initial_guess = mie(opt.pot_r, 1.0, 1.0)  # 1-D array of potential values.
alphabet = ['A', 'B', 'C']
rdf_targets = [np.loadtxt('rdfs/C3-C3-state_{0}.txt'.format(i))
                for i in alphabet]

pair0 = Pair('C3', 'C3', initial_guess)
alphas = [1.0, 1.0, 1.0]

### Add targets to pair

Loops through each `state`, `target`, and `alpha` in `zip`. Adds the appropriate states, and converts `pair0` into a list for the `optimize()` function.

In [ ]:
for state, target, alpha in zip(states, rdf_targets, alphas):
    pair0.add_state(state, target, alpha, indices)
pairs = [pair0]

### Do magic

Sprinkle fairy dust over the code.


Calls the `optimize` function with the parameters given. 
Performs five iterations, with each successive iteration usually producing finer, better output. 
Uses the `hoomd` engine to run the simulations.

In [ ]:
opt.optimize(states, pairs, n_iterations=5, engine='hoomd')